Plan :
1. Option pricing using BSM model
2. Random Walk
3. Brownian Motion


**PS.** To know more about Options : https://www.investopedia.com/terms/b/blackscholes.asp

In [ ]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.stats import norm
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# BSM Parameters
r = .04 #interest free rate
S = 20 #stock price
K = 30 #strike price
T = 40/365 #time in years
sigma = .40 #voltatility 

def blackscholes(r,S,K,T,sigma):
    '''Calculate option (Call or Put) pricing based on given parameters'''
    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    try:        
        call = S*norm.cdf(d1, 0, 1) - K*np.exp(-r*T)*norm.cdf(d2, 0, 1)
        print(f'Call price is : {call}')
        put = -(S*norm.cdf(-d1, 0, 1) - K*np.exp(-r*T)*norm.cdf(-d2, 0, 1))
        print(f'Put price is : {put}')
        return call, put        
    except:
        print('Check all the parameters!!!')
        return None, None
    

In [ ]:
blackscholes(r,S,K,T,sigma)

# Random walk
1. Toss a coin
2. If H then get 1 dollar else lose 1 (H and T would arise with equal probability)
3. The gain (which could be negative) that you will be making is in fact a Random walk.

Above experiment was performed at discrete time steps, say at each second we tossed the coin. If we start dividing the time step to infinitesimal we will get Brownian motion.

In [ ]:
def random_walk():
    n = 100
#     n = 100
    d = 4 # number of random walks
#     n=100
    walk = np.zeros(shape=(1,d))
    for _ in range(n):
        walk = np.concatenate((walk, np.random.choice([1,-1], size=(1,d))), axis=0)
#     print(np.cumsum(walk))
    plt.figure(figsize=(20,15))
    plt.plot(range(n+1), np.cumsum(walk, axis=0))
random_walk()

# Brownian Motion

**CREDIT** https://www.youtube.com/watch?v=hqSnruUe3tA

In [ ]:
def brownian_motion(m):
    # we want to produce m number of BM's
    n = 10_000
    T = 1
    times = np.linspace(0, T, n)
    dt = times[1] - times[0]
#     print(dt)
    dB = np.sqrt(dt) * np.random.normal(size=(n-1, m))#difference of BM 
    B0 = np.zeros(shape=(1, m))
    B = np.concatenate((B0, np.cumsum(dB, axis=0)), axis=0)
    plt.figure(figsize=(20,15))
    plt.plot(times, B)
    return B
    
brownian_motion(100)

In [ ]:
def quadratic_variation(B):
    '''Getting QV for a given BM B'''
    n = 10_000
    T = 1
    times = np.linspace(0, T, n)
    plt.plot(times, np.cumsum(np.power(np.diff(B,axis=0,prepend=0), 2), axis=0))
    plt.axhline(1)
quadratic_variation(brownian_motion(9))

In [ ]:
mu = 1
n = 50
dt = 0.1
x0 = 100
np.random.seed(1)

sigma = np.arange(0.8, 2, 0.2)

x = np.exp(
    (mu - sigma ** 2 / 2) * dt
    + sigma * np.random.normal(0, np.sqrt(dt), size=(len(sigma), n)).T
)
x = np.vstack([np.ones(len(sigma)), x])
x = x0 * x.cumprod(axis=0)

plt.plot(x)
plt.legend(np.round(sigma, 2))
plt.xlabel("$t$")
plt.ylabel("$x$")
plt.title(
    "Realizations of Geometric Brownian Motion with different variances\n $\mu=1$"
)

# Call and put option values
Proposition 5.6 (Sean Dineen) : 
1. r - interest rate
2. S, k - Stock value at time 0, Strike price at time T
3. Su , Sd are Stock value at time T; $0< d<1<e^{rT}<u$.
Then we calculate,
1. Risk neutral probability $p =\frac{e^{rT}-d}{u-d}$
2. Arbitrage-free price of Call option is $C_T=\frac{Su-k}{u-d}*(1-e^{-rT}*d)$
3. The seller's portfolio for hedging the call option consists of $\Delta=\frac{Su-k}{Su-Sd}$ shares and borrowings of $B=de^{-rT}*\left(\frac{Su-k}{u-d}\right)$ bonds.
4.  Arbitrage-free price of Put option is $P_T=\frac{Sd-k}{u-d}*(1-e^{-rT}*u)$

In [ ]:
def call_put_value(r,S,k,u,d,T):
    p = (np.exp(r*T) - d)/(u - d)
    C_T = (S*u - k)*(1 - np.exp(-r*T) * d) / (u - d)
    delta = (S*u - k) / (S*u - S*d)
    B = d*np.exp(-r*T)*(S*u - k)/(u - d)
    P_T = (S*d - k)*(1 - np.exp(-r*T) * u) / (u - d)
    return p, delta, B, C_T, P_T

In [ ]:
call_put_value(12/100, 20, 22, 24/20, 19/20, .25)

In [ ]:
call_put_value(12/100, 24, 22, 26/24, 21/24, .25)

In [ ]:
call_put_value(12/100, 19, 22, 21/19, 17/19, .25)